In [1]:
import json
import pygraphviz as pgv
import os
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [3]:
class ConversationState:
    def __init__(self, name=None, parent=None):
        self.name = name
        self.parent = parent
        self.messages = []
        self.to = {}
        self.children = []

    def add_message(self, message):
        self.messages.append(message)

    def add_transition(self, trigger, next_state):
        self.to[trigger] = next_state

    def add_child(self, child_state):
        self.children.append(child_state)
        child_state.parent = self

    def get_next_state(self, response):
        if response in self.to:
            return self.to[response]
        elif self.parent:
            return self.parent.get_next_state(response)
        else:
            return None

    def get_root(self):
        if self.parent:
            return self.parent.get_root()
        else:
            return self

    def get_hierarchy_path(self):
        if self.parent and self.parent.name != "root":
            return self.parent.get_hierarchy_path() + "_" + self.name
        else:
            return self.name

In [4]:
def initialize_conversation_states(json_data):
    def create_state(state_data, parent=None):
        state = ConversationState(name=state_data["name"], parent=parent)
        state.messages = state_data.get("messages", [])

        for child_data in state_data.get("children", []):
            child_state = create_state(child_data, parent=state)
            state.add_child(child_state)

        return state

    root_state = create_state(json_data)
    return root_state

In [5]:
def apply_transitions(root_state, transitions_data):
    state_map = {}

    def traverse_and_map_states(state):
        state_map[state.get_hierarchy_path()] = state
        for child in state.children:
            traverse_and_map_states(child)

    def find_state_by_path(path):
        return state_map.get(path)

    traverse_and_map_states(root_state)

    for transition in transitions_data:
        trigger = transition["trigger"]
        source_paths = transition["source"]
        dest_path = transition["dest"]

        if not isinstance(source_paths, list):
            source_paths = [source_paths]

        for source_path in source_paths:
            source_path = source_path
            source_state = find_state_by_path(source_path)
            dest_state = find_state_by_path(dest_path)

            if source_state and dest_state:
                source_state.add_transition(trigger, dest_state)
            else:
                print(f"Warning: Invalid transition - Source: {source_path}, Destination: {dest_path}")

In [6]:
def visualize_conversation_states(root_state, transitions_data):
    graph = pgv.AGraph(directed=True)

    graph.graph_attr['fontname'] = 'Consolas'
    graph.node_attr['fontname'] = 'Consolas'
    graph.node_attr['shape'] = 'box'
    graph.node_attr['style'] = 'rounded'
    graph.edge_attr['fontname'] = 'Consolas'

    def add_state_to_graph(state, parent_subgraph=None):
        if parent_subgraph is None:
            subgraph = graph
        else:
            subgraph = parent_subgraph.add_subgraph(name=f"cluster_{state.get_hierarchy_path()}")
            subgraph.graph_attr['style'] = 'rounded'

        if not (parent_subgraph is None):
            subgraph.add_node(state.get_hierarchy_path(), label=state.name)

        for child in state.children:
            add_state_to_graph(child, subgraph)

    add_state_to_graph(root_state)

    for transition in transitions_data:
        trigger = transition["trigger"]
        source_paths = transition["source"]
        dest_path = transition["dest"]

        if not isinstance(source_paths, list):
            source_paths = [source_paths]

        for source_path in source_paths:
            source_state = graph.get_node(source_path)
            dest_state = graph.get_node(dest_path)

            if source_state and dest_state:
                graph.add_edge(source_state, dest_state, label=trigger)

    graph.layout(prog='dot')
    
    if not os.path.exists(os.environ.get("OUTPUT_DIR")):
        os.makedirs(os.environ.get("OUTPUT_DIR"))
        
    graph.draw(os.path.join(os.environ.get("OUTPUT_DIR"), 'state_diagram.png'))

In [7]:
def print_state_hierarchy(state, level=0):
    print("  " * level + state.get_hierarchy_path())
    for child in state.children:
        print_state_hierarchy(child, level + 1)

In [8]:
with open(os.path.join(os.environ.get("INPUT_DIR"), "states.json")) as file:
    json_data = json.load(file)

with open(os.path.join(os.environ.get("INPUT_DIR"), "transitions.json")) as file:
    transitions_data = json.load(file)

In [9]:
root_state = initialize_conversation_states(json_data)


print_state_hierarchy(root_state)

apply_transitions(root_state, transitions_data)

visualize_conversation_states(root_state, transitions_data)

root
  start
    start_select-ready
  notready
    notready_select-get-set-state
      notready_select-get-set-state_get-state
        notready_select-get-set-state_get-state_select-tool
          notready_select-get-set-state_get-state_select-tool_compose-question
          notready_select-get-set-state_get-state_select-tool_compose-python
          notready_select-get-set-state_get-state_select-tool_compose-powershell
          notready_select-get-set-state_get-state_select-tool_compose-screenshot
      notready_select-get-set-state_set-state
        notready_select-get-set-state_set-state_select-tool
          notready_select-get-set-state_set-state_select-tool_compose-message
          notready_select-get-set-state_set-state_select-tool_compose-python
          notready_select-get-set-state_set-state_select-tool_compose-powershell
          notready_select-get-set-state_set-state_select-tool_compose-screenshot
    notready_select-ready
  ready
    ready_select-tool
      ready_sele